# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 參考範例，將鐵達尼的船票票號( 'Ticket' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df_o = df[object_features]
df_o = df_o.fillna('None')
train_num = train_Y.shape[0]
df_o.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 承上題，三者比較效果何者最好?

In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_o_temp = pd.DataFrame()
for c in df_o.columns:
    df_o_temp[c] = LabelEncoder().fit_transform(df_o[c])
df_n_temp = df.drop(df_o.columns,axis=1)
df_temp = pd.merge(df_n_temp,df_o_temp,left_index=True, right_index=True, how='outer')
# 數值變數補中位數
df_temp = df_temp.fillna(df_temp.median())
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.6116516247503802


,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Cabin,Embarked
0,3.0,22.0,1.0,0.0,7.2500,155,1,720,185,3
0,3.0,34.5,0.0,0.0,7.8292,155,1,720,185,3
1,1.0,38.0,1.0,0.0,71.2833,286,0,816,106,0
1,3.0,47.0,1.0,0.0,7.0000,286,0,816,106,0
2,3.0,26.0,0.0,0.0,7.9250,523,0,914,185,3


In [4]:
# 加上 'Cabin' 欄位的計數編碼
count_df = df_o.groupby("Cabin")['Name'].agg({"Cabin_Count":"size"}).reset_index()
df_o = pd.merge(df_o, count_df, on=['Cabin'], how='left')
count_df.sort_values(by=['Cabin'], ascending=False).head(10)

,Cabin,Cabin_Count
186,T,1
185,None,1014
184,G6,5
183,F4,4
182,F38,1
181,F33,4
180,F2,4
179,F G73,2
178,F G63,2
177,F E69,1


In [5]:
df_o.head()

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014


In [6]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_o_temp = df_o.drop(["Cabin"],axis=1)
df_n_temp = df.drop(object_features,axis=1)
df_temp = pd.merge(df_n_temp,df_o_temp,left_index=True, right_index=True, how='outer')
# # 數值變數補中位數
# df_temp = df_temp.fillna(df_temp.median())
# train_X = df_temp[:train_num]
# estimator = LogisticRegression()
# print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
# df_temp.head()

In [7]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_o_temp = pd.DataFrame()
for c in object_features:
    df_o_temp[c] = LabelEncoder().fit_transform(df_o[c])
df_o_temp['Cabin_Count'] = df_o['Cabin_Count']
## 
df_o_temp = df_o_temp.drop(["Cabin"],axis=1)
df_n_temp = df.drop(object_features,axis=1)
df_temp = pd.merge(df_n_temp,df_o_temp,left_index=True, right_index=True, how='outer')
# 數值變數補中位數
df_temp = df_temp.fillna(df_temp.median())
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.60828711530339


,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Embarked,Cabin_Count
0,3.0,22.0,1.0,0.0,7.2500,155,1,720,3,1014
0,3.0,34.5,0.0,0.0,7.8292,155,1,720,3,1014
1,1.0,38.0,1.0,0.0,71.2833,286,0,816,0,2
1,3.0,47.0,1.0,0.0,7.0000,286,0,816,0,2
2,3.0,26.0,0.0,0.0,7.9250,523,0,914,3,1014


In [8]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_o_temp = pd.DataFrame()
for c in object_features:
    df_o_temp[c] = LabelEncoder().fit_transform(df_o[c])
df_o_temp['Cabin_Hash'] = df_o['Cabin'].map(lambda x:hash(x) % 10)

## 
df_o_temp = df_o_temp.drop(["Cabin"],axis=1)
df_n_temp = df.drop(object_features,axis=1)
df_temp = pd.merge(df_n_temp,df_o_temp,left_index=True, right_index=True, how='outer')
# 數值變數補中位數
df_temp = df_temp.fillna(df_temp.median())
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.6038368146246507


,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Embarked,Cabin_Hash
0,3.0,22.0,1.0,0.0,7.2500,155,1,720,3,3
0,3.0,34.5,0.0,0.0,7.8292,155,1,720,3,3
1,1.0,38.0,1.0,0.0,71.2833,286,0,816,0,9
1,3.0,47.0,1.0,0.0,7.0000,286,0,816,0,9
2,3.0,26.0,0.0,0.0,7.9250,523,0,914,3,3


In [9]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_o_temp = pd.DataFrame()
for c in object_features:
    df_o_temp[c] = LabelEncoder().fit_transform(df_o[c])
df_o_temp['Cabin_Hash'] = df_o['Cabin'].map(lambda x:hash(x) % 10)
df_o_temp['Cabin_Count'] = df_o['Cabin_Count']

## 
df_o_temp = df_o_temp.drop(["Cabin"],axis=1)
df_n_temp = df.drop(object_features,axis=1)
df_temp = pd.merge(df_n_temp,df_o_temp,left_index=True, right_index=True, how='outer')
# 數值變數補中位數
df_temp = df_temp.fillna(df_temp.median())
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.6060462722893608


,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Embarked,Cabin_Hash,Cabin_Count
0,3.0,22.0,1.0,0.0,7.2500,155,1,720,3,3,1014
0,3.0,34.5,0.0,0.0,7.8292,155,1,720,3,3,1014
1,1.0,38.0,1.0,0.0,71.2833,286,0,816,0,9,2
1,3.0,47.0,1.0,0.0,7.0000,286,0,816,0,9,2
2,3.0,26.0,0.0,0.0,7.9250,523,0,914,3,3,1014


In [10]:
# 'Cabin'獨熱編碼 +其他類別 : 標籤編碼  + 邏輯斯迴歸
df_Cabin_onehot = pd.get_dummies(df_o["Cabin"])
## 其他類別  
df_o_temp = pd.DataFrame()
for c in df_o.columns:
    df_o_temp[c] = LabelEncoder().fit_transform(df_o[c])
    
df_o_temp = pd.merge(df_o_temp, df_Cabin_onehot, left_index=True, right_index=True, how='outer')
df_o_temp = df_o_temp.drop(["Cabin","Cabin_Count"],axis=1)

df_n_temp = df.drop(object_features,axis=1)
df_temp = pd.merge(df_n_temp,df_o_temp,left_index=True, right_index=True, how='outer')
# 數值變數補中位數
df_temp = df_temp.fillna(df_temp.median())
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.6049351599961273


,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Embarked,A10,...,F E69,F G63,F G73,F2,F33,F38,F4,G6,None,T
0,3.0,22.0,1.0,0.0,7.2500,155,1,720,3,0,...,0,0,0,0,0,0,0,0,1,0
0,3.0,34.5,0.0,0.0,7.8292,155,1,720,3,0,...,0,0,0,0,0,0,0,0,1,0
1,1.0,38.0,1.0,0.0,71.2833,286,0,816,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3.0,47.0,1.0,0.0,7.0000,286,0,816,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.0,26.0,0.0,0.0,7.9250,523,0,914,3,0,...,0,0,0,0,0,0,0,0,1,0


> 結果: 鐵達尼生存預測，『Cabin』採標籤編碼 比 計數編碼，雜湊編碼，及獨熱編碼 在羅吉斯回歸預測上的結果好。